# Estimate Metalinks

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import muon as mu
import liana as li

/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from liana.funcomics import neg_to_zero

In [3]:
import decoupler as dc

Test object

In [4]:
adata = sc.datasets.pbmc68k_reduced()
adata = adata.raw.to_adata()

## Metabolite Enrichment

In [5]:
resource = pd.read_csv("liana/resource/metalinks_resource.csv")

In [6]:
resource = resource[['HMDB', 'Symbol']].rename(columns={'HMDB': 'ligand', 'Symbol': 'receptor'})

In [7]:
resource

,ligand,receptor
0,HMDB0000112,GABRB2
1,HMDB0001043,CLCN2
2,HMDB0015180,CLCN2
3,HMDB0000216,ADRA2B
4,HMDB0000068,ADRA2B
...,...,...
10160,HMDB0041130,MUSK
10161,HMDB0002496,MUSK
10162,HMDB0015397,MUSK
10163,HMDB0006284,MUSK


Process net

In [8]:
# replace direction with 1 or -1
met_net = pd.read_csv("liana/resource/metabolite_sets.csv")
met_net['direction'].replace({'degrading': -1, 'producing': 1}, inplace=True)

In [9]:
# NOTE: there are duplicates, Elias should append Sources
met_net = met_net.drop_duplicates(['HMDB', 'GENE', 'direction'])

In [10]:
# NOTE: there are sign-duplicates...
met_net = met_net.groupby(['HMDB', 'GENE']).aggregate({'direction': 'mean'}).reset_index()
# Remove any edge whose direction is not 1 or -1
met_net = met_net[met_net['direction'].isin([-1, 1])]

Metabolite Estimate

In [11]:
dc.run_ulm(adata, net=met_net, source='HMDB', target='GENE', weight='direction', use_raw=False, min_n=2)

In [12]:
met_est = li.fun.obsm_to_adata(adata, 'ulm_estimate')
met_est

AnnData object with n_obs × n_vars = 700 × 95
    obs: 'bulk_labels', 'n_genes', 'percent_mito', 'n_counts', 'S_score', 'G2M_score', 'phase', 'louvain'
    uns: 'bulk_labels_colors', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_umap', 'ulm_estimate', 'ulm_pvals'
    obsp: 'distances', 'connectivities'

To MuData

In [ ]:
mdata = mu.MuData({'metabolite':met_est, 'rna':adata})

Run LIANA

In [ ]:
li.mt.rank_aggregate(mdata, groupby='bulk_labels',
                     multi_kwargs=dict(x_mod='metabolite', y_mod='rna',
                                       x_transform = neg_to_zero,
                                       y_transform = False),
                     n_perms=None, use_raw=False, verbose=True,
                     resource=resource)

In [ ]:
mdata.uns['liana_res']

## Check Transporters

Transporter estimate

(Note worth it for this part)

In [ ]:
t_net = pd.read_csv("liana/resource/transporter_sets.csv")

In [ ]:
# when revirsible == True, else False
t_net['reversibility'] = t_net['reversibility']=='reversible'

In [ ]:
in_net = t_net[(t_net['transport_direction'] == 'in') | (t_net['reversibility'])]
in_net

In [ ]:

out_net = t_net[(t_net['transport_direction'] == 'out') | (t_net['reversibility'])]
out_net

In [ ]:
dc.run_ulm(adata, net=out_net, source='HMDB', target='GENE', weight='direction', use_raw=False, min_n=3)